In [2]:
answers = [
    [3, 0],
    [5, 1],
]

In [5]:
def alex(opponent_last, memory=None):
    return 1, memory;

def bob(opponent_last, memory=None):
    return 0, memory;

def clara(opponent_last, memory=None):
    if opponent_last is None:
        return 1, memory
    return opponent_last, memory

def denis(opponent_last, memory=None):
    if opponent_last is None:
        return 1, memory
    return 1 if opponent_last == 0 else 0, memory

def emma(opponent_last, memory=None):
    if memory is None:
        memory = {'round': 0}
    
    memory['round'] += 1
    
    if memory['round'] % 20 == 0:
        return 0, memory
    else:
        return 1, memory

def frida(opponent_last, memory=None):
    if memory is None:
        memory = {'opponent_always_1': True}
    
    if opponent_last is None:
        return 1, memory
    
    if opponent_last == 0:
        memory['opponent_always_1'] = False
    
    if memory['opponent_always_1']:
        return 1, memory
    else:
        return 0, memory

def george(opponent_last, memory=None):
    if memory is None:
        memory = { 'count': 0 }
        return 0, memory
    
    if opponent_last == 1:
        memory['count'] += 1
    
    if memory['count'] >= 10 and opponent_last == 1:
        memory['count'] = 0
        return 0, memory
    else:
        return opponent_last, memory

In [6]:
def play_game(strategy_a, strategy_b):
    score_a = 0
    score_b = 0
    last_a = None
    last_b = None
    memory_a = None
    memory_b = None

    max_dominant_a = 0
    max_dominant_b = 0
    current_dominant_a = 0
    current_dominant_b = 0
    
    for i in range(200):
        choice_a, memory_a = strategy_a(last_b, memory_a)
        choice_b, memory_b = strategy_b(last_a, memory_b)
        
        points_a = answers[choice_a][choice_b]
        points_b = answers[choice_b][choice_a]
        
        score_a += points_a
        score_b += points_b
        
        last_a = choice_a
        last_b = choice_b

        if points_a == 5 and points_b == 0:
            current_dominant_a += 1
            current_dominant_b = 0
            max_dominant_a = max(max_dominant_a, current_dominant_a)
        elif points_b == 5 and points_a == 0:
            current_dominant_b += 1
            current_dominant_a = 0
            max_dominant_b = max(max_dominant_b, current_dominant_b)
        else:
            current_dominant_a = 0
            current_dominant_b = 0
    
    return score_a, score_b, max_dominant_a, max_dominant_b

In [7]:
strategies = [
    ("alex", alex),
    ("bob", bob),
    ("clara", clara),
    ("denis", denis),
    ("emma", emma),
    ("frida", frida),
    ("george", george)
]

In [10]:
total_scores = {name: 0 for name, _ in strategies}
dominant_series = {name: 0 for name, _ in strategies}
pairwise_scores = {}
pairwise_dominant = {}

for i, (name1, strat1) in enumerate(strategies):
    for j, (name2, strat2) in enumerate(strategies):
            score1, score2, dom1, dom2 = play_game(strat1, strat2)
            total_scores[name1] += score1
            total_scores[name2] += score2
            dominant_series[name1] += dom1
            dominant_series[name2] += dom2
            
            pairwise_scores[(name1, name2)] = (score1, score2)
            pairwise_dominant[(name1, name2)] = (dom1, dom2)

In [11]:
print("\n" + "=" * 70)
print("ТАБЛИЦА 1: ОБЩЕЕ ЧИСЛО ОЧКОВ")
print("=" * 70)

print(f"{'':<12}", end="")
for name, _ in strategies:
    print(f"{name:<8}", end="")
print("Total")

for name1, _ in strategies:
    print(f"{name1:<12}", end="")
    for name2, _ in strategies:
        score1, score2 = pairwise_scores[(name1, name2)]
        print(f"{score1:<8}", end="")
    print(f"{total_scores[name1]:<8}")

print("\n" + "=" * 70)
print("ТАБЛИЦА 2: ДЛИНА НАИБОЛЬШЕЙ ДОМИНИРУЮЩЕЙ СЕРИИ")
print("=" * 70)

print(f"{'':<12}", end="")
for name, _ in strategies:
    print(f"{name:<8}", end="")
print("Total")

for name1, _ in strategies:
    print(f"{name1:<12}", end="")
    for name2, _ in strategies:
        dom1, dom2 = pairwise_dominant[(name1, name2)]
        print(f"{dom1:<8}", end="")
    print(f"{dominant_series[name1]:<8}")

print("\n" + "=" * 70)
print("РЕЙТИНГ СТРАТЕГИЙ ПО ОБЩЕМУ ЧИСЛУ ОЧКОВ")
print("=" * 70)
sorted_by_score = sorted(total_scores.items(), key=lambda x: x[1], reverse=True)
for rank, (name, score) in enumerate(sorted_by_score, 1):
    print(f"{rank:2}. {name:<10}: {score:>6} очков")

print("\n" + "=" * 70)
print("РЕЙТИНГ СТРАТЕГИЙ ПО ДОМИНИРУЮЩИМ СЕРИЯМ")
print("=" * 70)
sorted_by_dominant = sorted(dominant_series.items(), key=lambda x: x[1], reverse=True)
for rank, (name, dom) in enumerate(sorted_by_dominant, 1):
    print(f"{rank:2}. {name:<10}: {dom:>6} серий")


ТАБЛИЦА 1: ОБЩЕЕ ЧИСЛО ОЧКОВ
            alex    bob     clara   denis   emma    frida   george  Total
alex        200     1000    200     996     240     200     280     6232    
bob         0       600     597     0       30      597     600     4848    
clara       200     602     200     450     231     200     593     4952    
denis       1       1000    450     400     40      989     474     6708    
emma        190     980     226     940     220     901     305     7524    
frida       200     602     200     9       51      200     599     3722    
george      180     600     588     419     210     599     600     6392    

ТАБЛИЦА 2: ДЛИНА НАИБОЛЬШЕЙ ДОМИНИРУЮЩЕЙ СЕРИИ
            alex    bob     clara   denis   emma    frida   george  Total
alex        0       200     0       199     1       0       1       802     
bob         0       0       0       0       0       0       0       0       
clara       0       1       0       1       1       0       1       8       
deni